# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118967e+02     1.644150e+00
 * time: 0.08798599243164062
     1     1.068239e+01     8.790500e-01
 * time: 0.91485595703125
     2    -1.178945e+01     1.047460e+00
 * time: 1.0623548030853271
     3    -3.383072e+01     7.624953e-01
 * time: 1.2353599071502686
     4    -4.707583e+01     5.756535e-01
 * time: 1.3783400058746338
     5    -5.667652e+01     2.224638e-01
 * time: 1.5311999320983887
     6    -5.958685e+01     1.728719e-01
 * time: 1.6474158763885498
     7    -6.079381e+01     6.720891e-02
 * time: 1.757091999053955
     8    -6.131370e+01     4.101222e-02
 * time: 1.8726179599761963
     9    -6.160529e+01     3.232922e-02
 * time: 1.98087477684021
    10    -6.176836e+01     3.021242e-02
 * time: 2.097146987915039
    11    -6.192312e+01     2.258391e-02
 * time: 2.206707000732422
    12    -6.201802e+01     1.783924e-02
 * time: 2.3219778537750244
    13    -6.206929e+01     1.342226e-02
 * time: 2.430875778198242


In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557668
    AtomicNonlocal      14.8522640
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485367 
    Xc                  -19.3336818

    total               -62.261666462904
